In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

from utils import mnist, plot_graphs, mnist_transform
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader

In [2]:
train_loader, valid_loader, test_loader = mnist(valid=10000)

train_length = 1000
train_data = datasets.MNIST('./MNIST_data', train=True, download=True, transform=mnist_transform)
train_loader = DataLoader(train_data, batch_size=50, sampler=SubsetRandomSampler( np.arange(train_length) ))

In [3]:
class Net(nn.Module):
    def __init__(self, batchnorm=False, dropout=False, lr=1e-4, l2=0.):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 10)
        if batchnorm:
            self.bn = nn.BatchNorm1d(128)
        self.batchnorm = batchnorm
        
        self.dropout = dropout
        self.optim = optim.Adam(self.parameters(), lr=lr, weight_decay=l2)
        
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = torch.tanh(self.fc1(x))
        if self.batchnorm:
            x = self.bn(x)
        x = torch.tanh(self.fc2(x))
        if self.dropout:
            x = F.dropout(x, 0.5)
        x = self.fc3(x)
        x = torch.log_softmax(x, dim=1)
        return x
    
    def loss(self, output, target, **kwargs):
        self._loss = F.nll_loss(output, target, **kwargs)
        return self._loss

In [4]:
def train(epoch, models):
    train_size = len(train_loader.sampler)
        
    for batch_idx, (data, target) in enumerate(train_loader):
        for k, model in models.items():
            model.optim.zero_grad()
            output = model(data)
            loss = model.loss(output, target)
            loss.backward()
            model.optim.step()
            
        if batch_idx % 200 == 0:
            line = 'Train Epoch: {} [{}/{} ({:.0f}%)]\tLosses '.format(
                epoch, batch_idx * len(data), train_size, 100. * batch_idx / len(train_loader))
            losses = ' '.join(['{}: {:.6f}'.format(k, m._loss.item()) for k, m in models.items()])
            print(line + losses)
            
    else:
        batch_idx += 1
        line = 'Train Epoch: {} [{}/{} ({:.0f}%)]\tLosses '.format(
            epoch, batch_idx * len(data), train_size, 100. * batch_idx / len(train_loader))
        losses = ' '.join(['{}: {:.6f}'.format(k, m._loss.item()) for k, m in models.items()])
        print(line + losses)

In [5]:
def test(models, loader, log=None):
    test_size = len(loader.sampler)
    
    avg_lambda = lambda l: 'Loss: {:.4f}'.format(l)
    acc_lambda = lambda c, p: 'Accuracy: {}/{} ({:.0f}%)'.format(c, test_size, p)
    line = lambda i, l, c, p: '{}: '.format(i) + avg_lambda(l) + '\t' + acc_lambda(c, p)

    test_loss = {k: 0. for k in models}
    correct = {k: 0. for k in models}
    
    with torch.no_grad():
        for data, target in loader:
            # output = {k: m(data) for m in models}
            for k, m in models.items():
                output = m(data)
                test_loss[k] += m.loss(output, target, reduction='sum').item() # sum up batch loss
                pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
                correct[k] += pred.eq(target.data.view_as(pred)).cpu().sum().item()
    
    for k in models:
        test_loss[k] /= test_size
    correct_pct = {k: 100. * correct[k] / test_size for k in correct}
    lines = '\n'.join([line(k, test_loss[k], correct[k], correct_pct[k]) for k in models]) + '\n'
    report = 'Test set:\n' + lines
    if log is not None:
        for k in models:
            log[k].append((test_loss[k], correct_pct[k]))
    print(report)

In [6]:
def plot_train_test(train_log, test_log):
    data = {}
    for name, model in train_log.items():
        train_name = '{}_train'.format(name)
        test_name = '{}_test'.format(name)
        data[train_name] = model
        data[test_name] = test_log[name]
    
    plot_graphs(data, 'loss')
    plot_graphs(data, 'accuracy')

In [ ]:
#models = {'default': Net(False, False), 'bn': Net(True, False), 'drop': Net(False, True), 'both': Net(True, True)}
models = {'default': Net(False, True, lr=1e-3)}
train_log = {k: [] for k in models}
test_log = {k: [] for k in models}

In [ ]:
%%time
for model in models.values():
    for g in model.optim.param_groups:
        g['lr'] = 1e-3

for epoch in range(0, 30):
    for model in models.values():
        model.train()
    train(epoch, models)
    for model in models.values():
        model.eval()
    test(models, train_loader, train_log)
    test(models, valid_loader, test_log)

Train Epoch: 0 [0/1000 (0%)]	Losses default: 2.341486
Train Epoch: 0 [1000/1000 (100%)]	Losses default: 1.085487
Test set:
default: Loss: 1.0592	Accuracy: 763.0/1000 (76%)

Test set:
default: Loss: 1.1474	Accuracy: 7134.0/10000 (71%)

Train Epoch: 1 [0/1000 (0%)]	Losses default: 1.134236
Train Epoch: 1 [1000/1000 (100%)]	Losses default: 0.561197
Test set:
default: Loss: 0.5820	Accuracy: 874.0/1000 (87%)

Test set:
default: Loss: 0.7110	Accuracy: 8105.0/10000 (81%)

Train Epoch: 2 [0/1000 (0%)]	Losses default: 0.384903
Train Epoch: 2 [1000/1000 (100%)]	Losses default: 0.405963
Test set:
default: Loss: 0.3809	Accuracy: 909.0/1000 (91%)

Test set:
default: Loss: 0.5465	Accuracy: 8450.0/10000 (84%)

Train Epoch: 3 [0/1000 (0%)]	Losses default: 0.627154
Train Epoch: 3 [1000/1000 (100%)]	Losses default: 0.318451
Test set:
default: Loss: 0.2862	Accuracy: 941.0/1000 (94%)

Test set:
default: Loss: 0.4794	Accuracy: 8626.0/10000 (86%)

Train Epoch: 4 [0/1000 (0%)]	Losses default: 0.221927
Train 

In [ ]:
plot_train_test(train_log, test_log)